## Examples with ```CopasiProcess``` using ``builder.Builder`` and ``biosimulator_processes.data_model``

In [4]:
import sys 

sys.path.insert(0, '..')

In [5]:
from builder import Builder, Process
from process_bigraph import pp, pf 
from biosimulator_processes import CORE
from biosimulator_processes.data_model import *
from biosimulator_processes.biosimulator_builder import BiosimulatorBuilder

CobraProcess registered successfully.
CopasiProcess registered successfully.
SmoldynProcess registered successfully.
TelluriumProcess registered successfully.


#### Initialize a fresh instance of the `BiosimulatorBuilder`, which is inherently configured by `biosimulator_processes.CORE`.

In [ ]:
b = BiosimulatorBuilder()

In [ ]:
###### list registered processes

b.list_processes()

Let's load a BIOMODEL from `basico` and inspect the parameters we can adjust:

#### Example 1: Add a simple `CopasiProcess`-based composite to the bigraph, specifying parameter values through the `data_model`. For now, let's try loading a biomodel and adding a change to initial concentration of a species. We will then add another copasi process instance, whose model is this time loaded from a file (to demonstrate the data model)

Let's define a model instance from a file.

In [ ]:
from basico import * 


model_filepath = '../biosimulator_processes/model_files/BIOMD0000000061_url.xml'

os.path.exists(model_filepath)

In [ ]:
model_from_file = load_model(model_filepath)
get_species(model=model_from_file)

In [ ]:
# 1. specify model changes
process_model_changes = ModelChanges(
    species_changes=[SpeciesChanges(name='ADP', initial_concentration=2.1**3)]
)


pp(process_model_changes.model_dump())

In [ ]:
# 2. define the model schema to be used by the composite process (one of the copasiprocess config parameters)

process_model_from_file = Model(
    model_id='copasi_process_from_file',
    model_source=model_filepath,
    model_name='Simple Composite Process from File',
    model_changes=process_model_changes
)


pp(process_model_from_file.model_dump())

In [ ]:
# 3. Define config schema to be used as 'config' parameter of Process constructor
process_config_from_file = CopasiProcessConfigSchema(
    method='stochastic',
    model=process_model_from_file,
    process_name='simple_copasi'
)


pp(process_config_from_file.model_dump())

In [ ]:
process_config_from_file.model

In [ ]:
# 4. Add the process instance by the name of 'simple_copasi' to the builder


b[process_config_from_file.process_name].add_process(
    name='CopasiProcess',
    model=process_config_from_file.model,
    method=process_config_from_file.method
)

Let's load a BIOMODEL from `basico` and inspect the parameters we can adjust, and then add it to the composite process:

In [1]:
from basico import *
from basico import biomodels


biomodel_id = 'BIOMD0000000861'
biomodel = load_biomodel(biomodel_id)
get_species(model=biomodel)

,compartment,type,unit,initial_concentration,initial_particle_number,initial_expression,expression,concentration,particle_number,rate,particle_number_rate,key,sbml_id
name,,,,,,,,,,,,,
EpoRJAK2,cyt,reactions,nmol/l,3.976220e+00,9.578143e+14,,,3.976220e+00,9.578143e+14,-0.314737,-7.581552e+13,Metabolite_0,EpoRJAK2
EpoRpJAK2,cyt,reactions,nmol/l,0.000000e+00,0.000000e+00,,,0.000000e+00,0.000000e+00,0.314737,7.581552e+13,Metabolite_1,EpoRpJAK2
p1EpoRpJAK2,cyt,reactions,nmol/l,0.000000e+00,0.000000e+00,,,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,Metabolite_2,p1EpoRpJAK2
p2EpoRpJAK2,cyt,reactions,nmol/l,0.000000e+00,0.000000e+00,,,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,Metabolite_3,p2EpoRpJAK2
p12EpoRpJAK2,cyt,reactions,nmol/l,0.000000e+00,0.000000e+00,,,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,Metabolite_4,p12EpoRpJAK2
EpoRJAK2_CIS,cyt,reactions,nmol/l,0.000000e+00,0.000000e+00,,,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,Metabolite_5,EpoRJAK2_CIS
SHP1,cyt,reactions,nmol/l,2.672510e+01,6.437693e+15,,,2.672510e+01,6.437693e+15,0.000000,0.000000e+00,Metabolite_6,SHP1
SHP1Act,cyt,reactions,nmol/l,0.000000e+00,0.000000e+00,,,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,Metabolite_7,SHP1Act
STAT5,cyt,reactions,nmol/l,7.975350e+01,1.921147e+16,,,7.975350e+01,1.921147e+16,0.000000,0.000000e+00,Metabolite_8,STAT5


In [2]:
reaction_names = get_reactions(model=biomodel)

reaction_names

,scheme,flux,particle_flux,function,key,sbml_id,mapping
name,,,,,,,
reaction_1,EpoRJAK2 -> EpoRpJAK2; Epo SOCS3,0.125895,7.581552e+13,Function for reaction_1,Reaction_0,reaction_1,"{'Epo': 'Epo', 'EpoRJAK2': 'EpoRJAK2', 'JAK2Ac..."
reaction_2,EpoRpJAK2 -> EpoRJAK2; SHP1Act,0.000000,0.000000e+00,Function for reaction_2,Reaction_1,reaction_2,"{'EpoRpJAK2': 'EpoRpJAK2', 'JAK2EpoRDeaSHP1': ..."
reaction_3,EpoRpJAK2 -> p1EpoRpJAK2; SOCS3,0.000000,0.000000e+00,Function for reaction_3,Reaction_2,reaction_3,"{'EpoRActJAK2': 'EpoRActJAK2', 'EpoRpJAK2': 'E..."
reaction_4,EpoRpJAK2 -> p2EpoRpJAK2; EpoRJAK2_CIS SOCS3,0.000000,0.000000e+00,Function for reaction_4,Reaction_3,reaction_4,"{'EpoRActJAK2': 'EpoRActJAK2', 'EpoRCISInh': '..."
reaction_5,p1EpoRpJAK2 -> p12EpoRpJAK2; EpoRJAK2_CIS SOCS3,0.000000,0.000000e+00,Function for reaction_5,Reaction_4,reaction_5,"{'EpoRActJAK2': 'EpoRActJAK2', 'EpoRCISInh': '..."
reaction_6,p2EpoRpJAK2 -> p12EpoRpJAK2; SOCS3,0.000000,0.000000e+00,Function for reaction_6,Reaction_5,reaction_6,"{'EpoRActJAK2': 'EpoRActJAK2', 'SOCS3': 'SOCS3..."
reaction_7,p1EpoRpJAK2 -> EpoRJAK2; SHP1Act,0.000000,0.000000e+00,Function for reaction_7,Reaction_6,reaction_7,"{'JAK2EpoRDeaSHP1': 'JAK2EpoRDeaSHP1', 'SHP1Ac..."
reaction_8,p2EpoRpJAK2 -> EpoRJAK2; SHP1Act,0.000000,0.000000e+00,Function for reaction_8,Reaction_7,reaction_8,"{'JAK2EpoRDeaSHP1': 'JAK2EpoRDeaSHP1', 'SHP1Ac..."
reaction_9,p12EpoRpJAK2 -> EpoRJAK2; SHP1Act,0.000000,0.000000e+00,Function for reaction_9,Reaction_8,reaction_9,"{'JAK2EpoRDeaSHP1': 'JAK2EpoRDeaSHP1', 'SHP1Ac..."


In [6]:
rparam = ReactionParameter(
    parameter_name='EpoRpRJAK2',
    value=21312.2
)

rparam2 = ReactionParameter(
    parameter_name='EpoRJAK2',
    value=21312.2
)

rc1 = ReactionChanges(
    reaction_name='reaction_1',
    parameter_changes=[rparam, rparam2]
)

In [7]:
pp(rc1.model_dump())

{ 'parameter_changes': [ {'parameter_name': 'EpoRpRJAK2', 'value': 21312.2},
                         {'parameter_name': 'EpoRJAK2', 'value': 21312.2}],
  'reaction_name': 'reaction_1',
  'reaction_scheme': None}


In [8]:
get_parameters(model=biomodel)

,type,unit,initial_value,initial_expression,expression,value,rate,key,sbml_id
name,,,,,,,,,
ActD,fixed,,0.000000e+00,,,0.000000e+00,0.0,ModelValue_0,ActD
CISEqc,fixed,,4.328710e+02,,,4.328710e+02,0.0,ModelValue_1,CISEqc
CISEqcOE,fixed,,5.302610e-01,,,5.302610e-01,0.0,ModelValue_2,CISEqcOE
CISInh,fixed,,7.846530e+08,,,7.846530e+08,0.0,ModelValue_3,CISInh
CISRNADelay,fixed,,1.447750e-01,,,1.447750e-01,0.0,ModelValue_4,CISRNADelay
CISRNAEqc,fixed,,1.000000e+00,,,1.000000e+00,0.0,ModelValue_5,CISRNAEqc
CISRNATurn,fixed,,1.000000e+03,,,1.000000e+03,0.0,ModelValue_6,CISRNATurn
CISTurn,fixed,,8.398420e-03,,,8.398420e-03,0.0,ModelValue_7,CISTurn
CISoe,fixed,,0.000000e+00,,,0.000000e+00,0.0,ModelValue_8,CISoe


In [9]:
param_changes = GlobalParameterChanges(
    name='ActD',
    initial_value=25.0,
    initial_expression='ActD + CISoe -> A'
)

pp(param_changes.model_dump())

{ 'expression': None,
  'initial_expression': 'ActD + CISoe -> A',
  'initial_value': 25.0,
  'name': 'ActD',
  'param_type': None,
  'status': None}


In [12]:
model_changes = ModelChanges(
    reaction_changes=[rc1],
    global_parameter_changes=[param_changes]
)

pp(model_changes.model_dump())

{ 'global_parameter_changes': [ { 'expression': None,
                                  'initial_expression': 'ActD + CISoe -> A',
                                  'initial_value': 25.0,
                                  'name': 'ActD',
                                  'param_type': None,
                                  'status': None}],
  'reaction_changes': [ { 'parameter_changes': [ { 'parameter_name': 'EpoRpRJAK2',
                                                   'value': 21312.2},
                                                 { 'parameter_name': 'EpoRJAK2',
                                                   'value': 21312.2}],
                          'reaction_name': 'reaction_1',
                          'reaction_scheme': None}],
  'species_changes': []}
